In [7]:
use std::io::Read;
use reqwest::Url;
use reqwest::blocking::Response;

fn main() {
    let client = reqwest::blocking::Client::new();
    let origin_url = "https://rolisz.ro/";
    let mut res = client.get(origin_url).send().unwrap();
    println!("Status for {}: {}", origin_url, res.status());

    let mut body  = String::new();
    res.read_to_string(&mut body).unwrap();
    println!("HTML: {}", &body[0..40]);
}


In [6]:
:dep reqwest = { version = "0.10", features = ["json", "blocking"] }

In [8]:
main()

Status for https://rolisz.ro/: 200 OK
HTML: <!DOCTYPE html>
<html lang="en">
<head


()

In [10]:
:dep tokio = { version = "0.2", features = ["full"] }
:dep quick-xml = { version = "*", features = [ "serialize" ] }
:dep serde = { version = "1.0", features = [ "derive" ] }
:dep rayon = { version = "*" }
:dep chrono = { version = "0.4", features = ["serde"] }
:dep select = "0.4.3"